<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
from __future__ import print_function

%matplotlib inline

import os
import numpy
import matplotlib.pyplot as plt
import matplotlib.animation

from IPython.display import HTML
from clawpack import pyclaw
from clawpack import riemann

# Finite Volume Methods for Nonlinear Scalar Conservation Laws

As we saw previously, if we have a nonlinear conservation law that it be considered in this form or a form of nonuniqueness will result apart from the use of the weak form.  This extends to the numerical case as well and is critical to come to proper approximation of the weak solutions to the conservation law.

## Godunov's Method

From our original formulation of Godunov's method we obtained the numerical flux by evaluating the solution along the grid cell edge in time
$$
    F^n_{i-1/2} = f(Q_{i-1/2}).
$$
For nonlinear scalar problems there are 5 different configurations that may occur:
1. Shock traveling to the left
1. Rarefaction traveling fully to the left
1. A centered rarefaction
1. Rarefaction traveling fully to the right
1. Shock traveling to the right

In all but case 3 we know what value to pick to evaluate the flux function, i.e. $Q_{i-1}$ for case 1 and 2 and $Q_i$ for case 4 and 5.  For case 2 however it is no longer clear what value $Q$ to use.

Suppose that $f''(q) > 0$ everywhere such that if $Q_{i-1} < Q_i$ leads to a rarefaction.  If $Q_{i-1} < q_s < Q_i$ such that $q_s$ is the unique value where $f'(q_s) = 0$.  The value $q_s$ is often called a **stagnation point** as it propagates with speed 0.  It can also be called a **sonic point** as in gas dynamics (and shallow water) the speeds can by 0 only if the fluid speed $|u|$ and the sound speed (or gravity wave speed) equal.

With this being said we can generalize the Godunov flux function as
$$
    F^n_{i-1/2} = \left \{ \begin{aligned}
        &f(Q_{i-1}) & & \text{if } Q_{i-1} > q_s &\text{and } & s > 0 \\
        &f(Q_{i}) & & \text{if } Q_{i} < q_s &\text{and } & s < 0 \\
        &f(q_s) & & \text{if } Q_{i-1} < q_s < Q_i & &
    \end{aligned} \right .
$$
where $s$ is the Rankine-Hugoniot derived shock speed.

## Fluctuations, Waves, and Speeds

We still want to express Godunov's method in terms of our waves and fluctuations.  Recall our standard, first-order scheme
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} (\mathcal{A}^+ \Delta Q_{i-1/2} + \mathcal{A}^- \Delta Q_{i+1/2} )
$$
where we define $\mathcal{A}^\pm \Delta Q_{i-1/2}$ as
$$\begin{aligned}
    &\mathcal{A}^+ \Delta Q_{i-1/2} = f(Q_i) - f(\widehat{Q}_{i-1/2}) \\
    &\mathcal{A}^- \Delta Q_{i-1/2} = f(\widehat{Q}_{i-1/2}) - f(Q_{i-1})
\end{aligned}$$

For the waves $\mathcal{W}_{i-1/2}$ and speeds $s_{i-1/2}$ that come into the higher-order correction terms we naturally have
$$
    \mathcal{W}_{i-1/2} = Q_i - Q_{i-1}
$$
and
$$
    s_{i-1/2} = \left \{ \begin{aligned}
        &\frac{f(Q_i) - f(Q_{i-1})}{Q_i - Q_{i-1}} & & \text{if } Q_{i-1} \neq Q_i \\
        &f'(Q_i) & & \text{if } Q_{i-1} = Q_i
    \end{aligned} \right .
$$

## Transonic Rarefactions and Entropy Fixes

Up until now we have only described a method that satisfies the Rankine-Hugoniot conditions but does not consider our previous case 3, a transonic/centered rarefaction.  If we were not to treat this case specially we would be violating our entropy conditions.

In order to handle this case we must apply what is mostly commonly termed an **entropy fix**.  An entropy fix aims to modify $\mathcal{A}^\pm \Delta Q_{i-1/2}$ such that it handles the transonic case and maintains an entropy satisfying solution.

In [ ]:
def true_solution(x, t):
    return numpy.zeros(x.shape)

def burgers_animation(order=2, efix=True):
   
    solver = pyclaw.ClawSolver1D(riemann.burgers_1D)

    solver.limiters = pyclaw.limiters.tvd.MC
    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap
    solver.order = order

    x = pyclaw.Dimension(-3.0, 3.0, 50, name='x')
    domain = pyclaw.Domain(x)
    num_eqn = 1
    state = pyclaw.State(domain, num_eqn)

    xc = domain.grid.x.centers
    state.q[0,:] = (xc < 0) * -numpy.ones(xc.shape) + 2.0 * (xc >= 0) * numpy.ones(xc.shape)
    state.problem_data['efix'] = efix

    claw = pyclaw.Controller()
    claw.tfinal = 6.0
    claw.num_output_times   = 30
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver

    claw.keep_copy = True
    claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    
    fig = plt.figure()
    axes = plt.subplot(1, 1, 1)
    axes.set_xlim((x[0], x[-1]))
    axes.set_ylim((-1.5, 2.5))
    axes.set_title("Burgers Equation")
    
    def init():
        axes.set_xlim((x[0], x[-1]))
        axes.set_ylim((-1.5, 2.5))
        computed_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'bo-')
        true_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'k-')
        return (computed_line, true_line)
    
    computed_line, true_line = init()
    
    def fplot(n):
        computed_line.set_data([x,], [claw.frames[n].q[0, :]])
        true_line.set_data([x,], [true_solution(x, claw.frames[n].t)])
        return (computed_line, true_line)

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=init, repeat=False)

In [ ]:
HTML(burgers_animation(order=1, efix=False).to_jshtml())

The practical application of an entropy fix usually consists of an indicator that the original solution may not be sufficient.  In other words in the case that there may exist a transonic rarefaction.  For scalar conservation laws this will occur when
$$
    f'(Q_{i-1}) < 0 < f'(Q_i)
$$
in which case we redefine $\mathcal{A}^\pm \Delta Q_{i-1/2}$ with
$$\begin{aligned}
    &\mathcal{A}^+ \Delta Q_{i-1/2} = f(Q_i) - f(q_s) \\
    &\mathcal{A}^- \Delta Q_{i-1/2} = f(q_s) - f(Q_i)
\end{aligned}$$

## Numerical Viscosity

It is evident that method mentioned above is not completely incorrect. How can this be the case when we know that the proper entropy satisfying solution should not be represented at all.  Instead we see a solution that partially does have the correct rarefaction structure.  What could be going on?

The fact that we are in reality getting a "pretty good" solution is that in reality the dominant error is providing a form of numerical viscosity that acts to create something like a physical viscosity that would match somewhat we want to approximate.  This suggests that, similar to finite-difference analysis, that what we really require is a form of regularization with some diffusion to obtain true solutions.

In the case of finite volume methods we can consider a numerical flux function
$$
    F_{i-1/2} = \frac{1}{2} [ f(Q_{i-1}) + f(Q_i) - |s_{i-1/2}| (Q_i - Q_{i-1})]
$$
that we know works as an unstable flux plus a diffusive like term.

## Lax-Friedrichs and Local Lax-Friedrichs Methods

A class of methods use a similar flux definition that was just mentioned perviously.

## Engquist-Osher method

## E-Schemes

## High-Resolution TVD Methods

## Conservation Form

## Lax-Wendroff Theorem

## Entropy Conditions Revisited

### Entropy Consistency of Godunov Methods

## Nonlinear Stability